## Exercise 1: Recipe Database

### 1.1 

Load the JSON recipes database provided in the data folder. (It has been zipped to save space.)

How many of the recipes are for breakfast food? 

Hint: The `description` would contain the work "breakfast".

In [1]:
# exercise 1.1

import pandas as pd
recipes = pd.read_json("C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-data-visualization-P2-main/ds-data-visualization-P2-main/data/recipes/recipe.json", compression='infer', lines = True) 
# - please read-up on the params. It will be important for future workshops

recipes_nonnan = recipes.dropna(subset='description')
print(f"There are {len(recipes_nonnan[recipes_nonnan['description'].str.contains('breakfast')])} breakfast recipes out of {len(recipes)} recipes.")

len(recipes_nonnan[recipes_nonnan['description'].str.contains('breakfast')])

There are 3442 breakfast recipes out of 173278 recipes.


3442

### 1.2 A simple recipe recommender

Let's build a recipe recommender: given a list of basic ingredients, find a recipe that uses all those ingredients.

Here is the list of ingredients that can be asked for:

```
['salt', 'pepper', 'oregano', 'sage', 'parsley',
 'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']
```

**Hint:** Build a new column for each of the ingredients that indicates whether that ingredient is in the recipe.

#### Example

```
recommend_ingredients(["parsley", "paprika", "tarragon"], df)

result: 
# The rows where these 3 ingredients are in the recipe
[2069, 74964, 93768, 113926, 137686, 140530, 158475, 158486, 163175, 165243]
```

In [2]:
recipes.columns

Index(['_id', 'name', 'ingredients', 'url', 'image', 'ts', 'cookTime',
       'source', 'recipeYield', 'datePublished', 'prepTime', 'description',
       'totalTime', 'creator', 'recipeCategory', 'dateModified',
       'recipeInstructions'],
      dtype='object')

In [3]:
recipes.sample(3)

,_id,name,ingredients,url,image,ts,cookTime,source,recipeYield,datePublished,prepTime,description,totalTime,creator,recipeCategory,dateModified,recipeInstructions
25224,{'$oid': '5161351596cc620d26155ee5'},"Chicken, Broccolini, Tomato, &amp; Artichoke ...",2 whole 2 whole\n3 Tablespoons 3 Tablespoons\n...,http://tastykitchen.com/recipes/main-courses/c...,http://static.tastykitchen.com/recipes/files/2...,{'$date': 1365325077917},PT12M,tastykitchen,4,2012-08-28,PT10M,"What a great way to inhale, I mean incorporate...",NaN,NaN,NaN,NaN,NaN
26122,{'$oid': '51613bd296cc620d26156267'},Bracioli (Stuffed Round Steak),4 pieces 4 pieces\n4 Tablespoons 4 Tablespoons...,http://tastykitchen.com/recipes/main-courses/b...,http://static.tastykitchen.com/recipes/files/2...,{'$date': 1365326802646},PT2H,tastykitchen,3,2010-09-16,PT5M,"This recipe is a piece of my childhood, taken ...",NaN,NaN,NaN,NaN,NaN
147156,{'$oid': '51ff32d196cc62205dfbbae2'},Blender Mayonnaise,\n\n\n\n\n,http://www.epicurious.com/recipes/food/views/B...,http://www.epicurious.com/images/recipesmenus/...,{'$date': 1375679185014},NaN,epicurious,Makes about 1 cup,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# exercise 1.2

def recommend_ingredients(ings):
    recipes_ing = recipes.dropna(subset='ingredients')
    for ingredient in ings:
        recipes_ing[ingredient] = recipes_ing['ingredients'].str.contains(ingredient)
        recipes_ing = recipes_ing[recipes_ing[ingredient]==True]
    return print(f"The rows where these {len(ings)} ingredients are in the recipe:\n{recipes_ing.index.tolist()}")

recommend_ingredients(["parsley", "paprika", "tarragon"])
recommend_ingredients(['salt', 'pepper', 'oregano', 'sage', 'parsley','paprika'])
recommend_ingredients(['salt', 'pepper', 'oregano', 'sage', 'parsley','rosemary', 'tarragon'])

The rows where these 3 ingredients are in the recipe:
[2069, 74964, 93768, 113926, 137686, 140530, 158475, 158486, 163175, 165243]
The rows where these 6 ingredients are in the recipe:
[103293, 122613, 126285, 127279]
The rows where these 7 ingredients are in the recipe:
[142515]


## Exercise 2: Movies!

Look at this [Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset) on Kaggle. Download it and use it for this question. 

The dataset is made up of several tables. The tables have common columns (`id` and `movie_id`) around which you can use to merge and join tables.

Your task is to find **The best director** in terms of average ratings of his movies. This can be from the `ratings` or `ratings_small` table, or simply the vote average in the `metadata` table. The director can be found in the `cast` table.

**💪You will have to use all of your skills to get this done, between using groupbys and merging multiple tables together.**

In [5]:
# exercise 2

# Use Pandas to assign CSV to each of the DataFrames. Will make it easier to organize your data

metadata_df = pd.read_csv("C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-data-visualization-P2-main/ds-data-visualization-P2-main/data/archive/movies_metadata.csv", low_memory=False)
credits_df = pd.read_csv("C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-data-visualization-P2-main/ds-data-visualization-P2-main/data/archive/credits.csv")
ratings_small_df = pd.read_csv("C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-data-visualization-P2-main/ds-data-visualization-P2-main/data/archive/ratings_small.csv")
ratings_df = pd.read_csv("C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-data-visualization-P2-main/ds-data-visualization-P2-main/data/archive/ratings.csv")

In [6]:
print(f"Metadata: {list(metadata_df.columns)}, \n\nCredits {list(credits_df.columns)}, \n\nRatings small {list(ratings_small_df.columns)}, \n\nRatings {list(ratings_df.columns)}")

Metadata: ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count'], 

Credits ['cast', 'crew', 'id'], 

Ratings small ['userId', 'movieId', 'rating', 'timestamp'], 

Ratings ['userId', 'movieId', 'rating', 'timestamp']


In [7]:
import ast

print(f"credits_df {len(credits_df)}")
print(f"metadata_df {len(metadata_df)}")
print(f"ratings_small_df {len(ratings_small_df)}")
print(f"ratings_df {len(ratings_df)}")

credits_df 45476
metadata_df 45466
ratings_small_df 100004
ratings_df 26024289


In [149]:
high_metadata_df = metadata_df[metadata_df['vote_average']==metadata_df['vote_average'].max()]
high_metadata_df['id'] = pd.to_numeric(high_metadata_df['id'])
high_metadata_df = high_metadata_df[['id','vote_average']]
high_metadata_df

C:\Users\Simona\AppData\Local\Temp\ipykernel_9624\1989623659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_metadata_df['id'] = pd.to_numeric(high_metadata_df['id'])


,id,vote_average
186,58372,10.0
394,278939,10.0
706,73183,10.0
738,255546,10.0
1634,64562,10.0
...,...,...
45047,405446,10.0
45198,412669,10.0
45251,28469,10.0
45362,173874,10.0


In [150]:
new_ratings_small_df = ratings_small_df.groupby('movieId', as_index=False).mean()
high_ratings_small_df = new_ratings_small_df[new_ratings_small_df['rating']==new_ratings_small_df['rating'].max()]
high_ratings_small_df = high_ratings_small_df.rename({'movieId':'id'}, axis=1)
high_ratings_small_df['id'] = pd.to_numeric(high_ratings_small_df['id'])
high_ratings_small_df = high_ratings_small_df[['id','rating']]
high_ratings_small_df

,id,rating
50,53,5.0
161,183,5.0
269,301,5.0
277,309,5.0
501,559,5.0
...,...,...
9027,156025,5.0
9053,160590,5.0
9061,161944,5.0
9063,162542,5.0


In [151]:
new_ratings_df = ratings_df.groupby('movieId', as_index=False).mean()
high_ratings_df = new_ratings_df[new_ratings_df['rating']==new_ratings_df['rating'].max()]
high_ratings_df = high_ratings_df.rename({'movieId':'id'}, axis=1)
high_ratings_df['id'] = pd.to_numeric(high_ratings_df['id'])
high_ratings_df = high_ratings_df[['id','rating']]
high_ratings_df

,id,rating
9320,27396,5.0
9568,27914,5.0
13479,66389,5.0
18102,90464,5.0
18697,92783,5.0
...,...,...
44879,175733,5.0
44881,175737,5.0
45039,176095,5.0
45090,176217,5.0


In [152]:
high_ratings_movies = pd.merge(high_ratings_df, high_ratings_small_df, on = 'id')
high_ratings_movies

,id,rating_x,rating_y
0,161944,5.0,5.0


In [153]:
high_ratings_movies2 = pd.merge(high_metadata_df, high_ratings_df, on = 'id')
high_ratings_movies2

,id,vote_average,rating


In [154]:
high_ratings_movies2 = pd.merge(high_metadata_df, high_ratings_small_df, on = 'id')
high_ratings_movies2

,id,vote_average,rating


In [156]:
def best_directors(df):
    director = []
    for i in df['id']:
        for value in credits_df['crew'][credits_df['id']==i]:
            ini_list = value
            res_list = ast.literal_eval(ini_list)
            for dictionary in res_list:
                if dictionary['job'] == 'Director':
                    x = dictionary['name']
                    y = dictionary['id']
                    director.append(x)
    return director

In [157]:
best_directors(high_ratings_movies)

[]

In [158]:
metadata_directors = best_directors(high_metadata_df)
ratings_directors = best_directors(high_ratings_df)
small_ratings_directors = best_directors(high_ratings_small_df)
print(len(metadata_directors), len(ratings_directors), len(small_ratings_directors))

195 17 90


In [159]:
common_directors = []
for director in ratings_directors:
    if director in metadata_directors:
        common_directors.append(director)
for director in ratings_directors:
    if director in small_ratings_directors:
        common_directors.append(director)

for director in metadata_directors:
    if director in small_ratings_directors:
        common_directors.append(director)
for director in metadata_directors:
    if director in ratings_directors:
        common_directors.append(director)

for director in small_ratings_directors:
    if director in ratings_directors:
        common_directors.append(director)
for director in small_ratings_directors:
    if director in metadata_directors:
        common_directors.append(director)
        
common_directors

['Stanley Donen', 'Stanley Donen']

In [160]:
def movie_director(director):
    movie_id = []
    for value in credits_df['crew']:
            ini_list = value
            res_list = ast.literal_eval(ini_list)
            for dictionary in res_list:
                if dictionary['name'] == director:
                    y = dictionary['id']
                    movie_id.append(y)
                    movie = value
    return credits_df[credits_df['crew']==movie]

In [161]:
best_movie = movie_director(common_directors[1])

In [162]:
best_movie

,cast,crew,id
42231,"[{'cast_id': 2, 'character': 'Alex', 'credit_i...","[{'credit_id': '52fe47ef9251416c750aa5dd', 'de...",19761


In [167]:
best_directors(best_movie)

['Stanley Donen']

In [169]:
print(f"The best director is {best_directors(best_movie)[0]}.")

The best director is Stanley Donen.
